# Initialization and some auxiliary functions

In [1]:
overlay_fname = "imported_design.bit"
#overlay_fname = "imported_design_cheribsd.bit"

#import ipdb # alternative to pdb that works in jupyter notebook (pip3 install ipdb)
from IPython.core.debugger import set_trace
import os, subprocess, sys, re, time, inspect, logging, random, json, math, glob, datetime
from pathlib import Path
from pynq import Overlay, allocate
#from pynq import GPIO
from threading import Thread, Lock
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
logger = logging.getLogger()
logger.setLevel(logging.DEBUG) # logging.INFO)

pynq_restarted = True

#BENCHMARK_TOOL_DIR = Path('/home/xilinx/benchmark_files/scripts/')
#sys.path.append(str(BENCHMARK_TOOL_DIR))
#import compare_classification_methods_2 as benchmark_ccm

PROGRAMS_DIR = Path('/home/xilinx/programs')

from dma_receiver import DmaReceiver
from bram_loader import Bram_Loader
from continuous_monitoring_system_controller import ContinuousMonitoringSystemController, BASIC_TRACE_FILTER_MODE
from riscv_instruction_decoder import get_riscv_instruction_name
from tcp_server import TCP_Server, get_my_ip
from console_io import Console_IO
import advanced_trace_filter
from anomaly_detection import Anomaly_Detection
from advanced_trace_filter import ATF_Watchpoints, ATF_MODE
from parse_objdump import parse_objdump
from packet_format import Packet_Format, DataFrame_Columns_Order
from sql_db import SQL_Barcodes_DB
from operational_config import Operational_Config

operational_config = Operational_Config('config.pickle')
operational_config.load()

TCP_SERVER_PORT = 9093
# tcp server for communicating with display (e.g. ESP3248S035C, but really any TCP client that connects)
tcp_server = TCP_Server(host_ip='0.0.0.0', port=TCP_SERVER_PORT)

BASE_DIR = Path('/home/xilinx/design_files')
OUTPUT_DIR = Path('/home/xilinx/output_files')
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

base = Overlay(str(BASE_DIR / overlay_fname))

bram_loader = Bram_Loader(base.PYNQ_wrapper_blocks.bram_loader.axi_gpio_2)
console_io = Console_IO(
    base.PYNQ_wrapper_blocks.console_io.axi_dma_console_io,
    recv_buffer_capacity=10000,
    send_buffer_capacity=10000
    )

# the long name is because of using hierarchy in Vivado block design
cms_ctrl_axi_gpio = base.PYNQ_wrapper_blocks.continuous_monitoring_system_blocks.axi_gpio_to_cms_ctrl_interface.axi_gpio_cms_ctrl.channel1    
cms_ctrl = ContinuousMonitoringSystemController(cms_ctrl_axi_gpio, verbose=True)



INPUT_BUFFER_DTYPE_SIZE_IN_BYTES = 8
#FIFO_SIZE = 32768
# +4 because DMA seems to have it's own buffer it fills before dma.recvchannel.transfer is even called
#buffer_length = min( base.PYNQ_wrapper_blocks.continuous_monitoring_system_blocks.axi_dma_0.buffer_max_size // ITEM_BYTE_SIZE, FIFO_SIZE)# + 4) 
#buffer_length = 4_000_000 // 8 # 4MB in total
#buffer_length = 16_000_000 // 8 # 16MB in total
#buffer_length = 10240*10 // 8 
buffer_length = 50_000_000 // INPUT_BUFFER_DTYPE_SIZE_IN_BYTES # 50MB in total
print('buffer_length =', buffer_length)

input_buffer = allocate(shape=(buffer_length,), dtype='u8')
input_buffer_2 = allocate(shape=(buffer_length,), dtype='u8')

dma_rec = base.PYNQ_wrapper_blocks.continuous_monitoring_system_blocks.axi_dma_0.recvchannel

# https://pynq.readthedocs.io/en/v2.7.0/_modules/pynq/lib/axigpio.html
gpio_rst_n_out = base.PYNQ_wrapper_blocks.axi_gpio_0.channel1[0]
gpio_rst_n_console_input = base.PYNQ_wrapper_blocks.axi_gpio_0.channel1[1]
gpio_rst_n_console_output = base.PYNQ_wrapper_blocks.axi_gpio_0.channel1[2]
#gpio_en_cpu_reset_server_request_put_out = base.axi_gpio_0.channel1[1]
#gpio_pc_stream_m_axis_tlast_interval = base.axi_gpio_1.channel1

gpio_fifo_wr_count = base.PYNQ_wrapper_blocks.axi_gpio_0.channel2[0:16]
gpio_fifo_rd_count = base.PYNQ_wrapper_blocks.axi_gpio_0.channel2[16:32]

# PERFORMANCE_EVENTS_FNAME = 'performance_event_names_selected.csv'
PERFORMANCE_EVENTS_FNAME = 'performance_event_names_used.csv'
with open(PERFORMANCE_EVENTS_FNAME) as f:    
    PERFORMANCE_EVENTS_COUNT = len(f.readlines()) - 1
print(f'Performance events count = {PERFORMANCE_EVENTS_COUNT}')
# PERFORMANCE_COUNTER_WIDTH = 7
# PERFORMANCE_COUNTERS_OVERFLOW_MAP_WIDTH = PERFORMANCE_EVENTS_COUNT
PERFORMANCE_COUNTER_WIDTH = 32
PERFORMANCE_COUNTERS_OVERFLOW_MAP_WIDTH = PERFORMANCE_EVENTS_COUNT
PC_WIDTH = 64
INSTR_WIDTH = 32
CLK_COUNTER_WIDTH = 64
FIFO_FULL_TICKS_COUNT_WIDTH = 64
GP_REGISTER_WIDTH = 128
AXI_DATA_WIDTH = 1024
FEATURE_EXTRACTOR_RESULT_WIDTH = 40
USED_AXI_BITS = sum([
    PERFORMANCE_EVENTS_COUNT*PERFORMANCE_COUNTER_WIDTH,
    PERFORMANCE_EVENTS_COUNT,
    PC_WIDTH,
    CLK_COUNTER_WIDTH,
    FIFO_FULL_TICKS_COUNT_WIDTH,
    INSTR_WIDTH,
    4*64, # A0 - A3
    FEATURE_EXTRACTOR_RESULT_WIDTH
    ])
CLK_SPEED = 50_000_000
# how many items from AXI can be stored in PYNQ allocated buffer
BUFFER_ITEM_CAPACITY = buffer_length // AXI_DATA_WIDTH * 8 # bytes / bits * bits_per_byte

# input buffer has "u8" dtype which has 8 bytes per element
# 16 elements are needed to store a single 1024-bit item from FIFO
# variable below can be used to know location of the end of transferred data in the input buffer
# so we can copy it and initiate another transfer
INPUT_BUFFER_LOCATIONS_PER_ITEM = AXI_DATA_WIDTH / 8 / INPUT_BUFFER_DTYPE_SIZE_IN_BYTES

# theoretically with 16MB allocated and 1024-bit items we could set TLAST_INTERVAL to 125000
#TLAST_INTERVAL = BUFFER_ITEM_CAPACITY - 5000
TLAST_INTERVAL = 0 # axilite_tap based tlast (setting tlast when receive transfer is requested)

def print_dma_channel_status(channel):
    print('dma.running =', channel.running)
    print('dma.idle =', channel.idle)
    print('dma.error =', channel.error)
    print('status =', hex(channel._mmio.read(channel._offset + 4)))
    
def reset_cpu(delay=0.001):
    ''' AXI GPIO controlled reset, active-low. '''
    #gpio_en_cpu_reset_server_request_put_out.write(0)
    gpio_rst_n_out.write(0)
    time.sleep(delay)
    gpio_rst_n_out.write(1)
    time.sleep(delay)
    #gpio_en_cpu_reset_server_request_put_out.write(1)
    #time.sleep(delay)
    #gpio_en_cpu_reset_server_request_put_out.write(0)
    #time.sleep(delay)
    
def print_fifo_data_counts():
    print('gpio_fifo_wr_count =', gpio_fifo_wr_count.read())
    print('gpio_fifo_rd_count =', gpio_fifo_rd_count.read())
    
def instr_to_strings(instructions_integers):
    ''' Requires riscv-python-model installed.
    If network connection is available, "python3 -m pip install riscv-model.
    If not, then on separate machine with internet:
        python3 -m pip download riscv-model -d .  
    Then copy the downloaded .whl file to pynq and install with:
        python3 -m pip install <file.whl> -f ./ --no-index   
    Usage:
        instr_to_string([0xB60006F, 0xFE0791E3])
        '''
    instructions_string = ' 0x'.join(f'{ii:08X}' for ii in instructions_integers)
    return os.popen(f'riscv-machinsn-decode hexstring {instructions_string}').read().strip().split('\n')


####################################################################
# 

def read_performance_event_names(f_name):
    ''' Reads events names from file, these were collected from CHERI-Flute source code by using this script:
    https://github.com/michalmonday/Flute/blob/continuous_monitoring/builds/RV64ACDFIMSUxCHERI_Flute_verilator/vcd/read_vcd.py
    '''
    with open(f_name) as f:
        return [line.strip().split(',')[2] for line in f.readlines()[1:]]

def pop_n_bits_value(val, n):
    ''' pop_n_bits_value(0xFFFF, 4) returns tuple like: (0xFFF, 0xF) '''
    bits_value = val & ((1<<n)-1)
    return val >> n, bits_value

# def parse_fifo_item(fifo_item):
#     ''' Parses a single fifo item (e.g. 1024 bits) numerical value. 
#         Single fifo item = {59bits padding, performance_counters805(7bits*115counters), instr32, clk_counter_delta64, pc64}
#         Padding is used because only power of 2s can be used as size in fifo generator block (or axi in general?)'''
#     perf_counters = []
#     for i in range(PERFORMANCE_EVENTS_COUNT):
#         fifo_item, perf_counter = pop_n_bits_value(fifo_item, PERFORMANCE_COUNTER_WIDTH)
#         perf_counters.append(perf_counter)
#     fifo_item, perf_counters_overflow_map = pop_n_bits_value(fifo_item, PERFORMANCE_COUNTERS_OVERFLOW_MAP_WIDTH)
#     fifo_item, pc = pop_n_bits_value(fifo_item, PC_WIDTH)
#     fifo_item, clk_counter = pop_n_bits_value(fifo_item, CLK_COUNTER_WIDTH)
#     fifo_item, instr = pop_n_bits_value(fifo_item, INSTR_WIDTH)
#     fifo_item, fifo_full_ticks_count = pop_n_bits_value(fifo_item, FIFO_FULL_TICKS_COUNT_WIDTH)
#     fifo_item, gp_reg_A0 = pop_n_bits_value(fifo_item, 64)
#     fifo_item, gp_reg_A1 = pop_n_bits_value(fifo_item, 64)
#     fifo_item, gp_reg_A2 = pop_n_bits_value(fifo_item, 64)
#     fifo_item, gp_reg_A3 = pop_n_bits_value(fifo_item, 64)
#     gp_regs = {'A0':gp_reg_A0, 'A1':gp_reg_A1, 'A2':gp_reg_A2, 'A3':gp_reg_A3}
#     return perf_counters, perf_counters_overflow_map, pc, clk_counter, instr, fifo_full_ticks_count, gp_regs

def parse_fifo_item(fifo_item, packet_format):
    ''' Parses a single fifo item (e.g. 1024 bits) numerical value. 
        Single fifo item = {59bits padding, performance_counters805(7bits*115counters), instr32, clk_counter_delta64, pc64}
        Padding is used because only power of 2s can be used as size in fifo generator block (or axi in general?)'''
    metrics_dict = {}
    for metric_name, bit_width in packet_format.items():
        fifo_item, metric_value = pop_n_bits_value(fifo_item, bit_width)
        metrics_dict[metric_name] = metric_value
    return metrics_dict

def get_dma_transfer(input_buffer, dma_rec=dma_rec, dont_wait=False, timeout_ms=None):
    ''' Returns the number of transferred items, each having 1024 bits. 
    This function relies on the hardware implementation that always delivers additional
    item to indicate end of transfer. (this allows to avoid situation where dma transfer
    hangs due to empty FIFO, that is why theres "-1" in return) 
    
    timeout_ms allows to repeat transfer until some value is received '''
    
    def get_dma_transfer_internal():
        ''' Helper function to avoid code repetition for repetitive transfers
        where timeout is used. items_transferred returned by it includes the ending item.
        So if FIFO is empty, this function returns 1. '''
        #print("get_dma_transfer_internal starting new transfer")
        dma_rec.transfer(input_buffer)
        if dont_wait:
            return 1
        #print("get_dma_transfer_internal waiting")
        dma_rec.wait()
        #print("get_dma_transfer_internal calculating transferred items")
        items_transferred = math.floor(dma_rec.transferred * 64 / AXI_DATA_WIDTH / 8)
        #print("get_dma_transfer_internal returning")
        return items_transferred
        
    items_transferred = 1
    # repetitive transfers when timeout_ms is used
    if timeout_ms is not None:
        timeout_s = timeout_ms / 1000
        start_time = time.time()
        while items_transferred <= 1 and time.time() - start_time < timeout_s:
            items_transferred = get_dma_transfer_internal()
            time.sleep(0.01)
        return items_transferred - 1
    
    # single transfer when timeout_ms is not used
    items_transferred = get_dma_transfer_internal()
    #print(f'items_transferred = {items_transferred}')
    return items_transferred - 1

def parse_input_buffer(input_buffer, items_transferred, packet_format=Packet_Format.data_pkt):
    ''' Function that parses the DMA receive buffer and returns a pandas DataFrame of all features like:
    - hardware performance events (with their overflow map)
    - program counters
    - number of clock ticks beween received items
    - instructions (in numerical form)
    - instructions decoded (in string form), this is slow so "dont_decode=True" may be used
    - number of clock ticks while internal trace storage was full and CPU was halted for that reason
      (allowing to measure performace decrease due to the use of this Continuous Monitoring System)
    - general purpose registers
    '''
    chunks_per_item = math.ceil(AXI_DATA_WIDTH/64)
    start = 0
    end = chunks_per_item
    #time_checkpoint = time.time()
    all_metrics = []
    for i in range(items_transferred):
        if i != 0:
            start += chunks_per_item
            end += chunks_per_item
        #time_checkpoint = time.time()
        fifo_item = int.from_bytes(bytes(input_buffer[start:end]), byteorder='little')
        #print(f'{time.time() - time_checkpoint}s')
        metrics_dict = parse_fifo_item(fifo_item, packet_format)
                                                    
        all_metrics.append(metrics_dict)
        
    df_metrics = pd.DataFrame(all_metrics)
    return df_metrics

def preprocess_df_metrics(df, dont_decode=False):
    '''df is the DataFrame returned by "parse_input_buffer"    
    Depending on available columns, this function may add some more columns like:
    - instr_names                     (dependent on 'instr')
    - instr_strings                   (dependent on 'instr')
    - clk_counter_halt_agnostic       (dependent on 'clk_counter' and 'fifo_full_ticks_count')
    - total_clk_counter_halt_agnostic (dependent on 'clk_counter_halt_agnostic')    '''
    if 'pc' in df.columns:
        # convert to hex string
        df['pc'] = df['pc'].apply(lambda x: f'{x:8X}')
    
    if 'instr' in df.columns:
        df['instr_names'] = df['instr'].apply(get_riscv_instruction_name)
        # df['instr_strings'] = '-' if dont_decode else instr_to_strings(df['instr'])
        # set all instr_strings to '-' if dont_decode=True
        instrs = df['instr']
        df['instr_strings'] = ['-'] * len(instrs) if dont_decode else instr_to_strings(instrs)
        df['instr'] = df['instr'].apply(lambda x: f'{x:08X}')

    if 'HPC_overflow_map' in df.columns:
        # convert to binary string with 39 ones and zeros
        # (it is assumed here that HPC overflow map has 39 bits, one for each HPC)
        df['HPC_overflow_map'] = df['HPC_overflow_map'].apply(lambda x: f'{x:08b}')

    if 'HPC_event_map' in df.columns:
        df['HPC_event_map'] = df['HPC_event_map'].apply(lambda x: f'{x:08b}')
        
    if 'clk_counter' in df.columns and 'fifo_full_ticks_count' in df.columns:
        df['clk_counter_halt_agnostic'] = df['clk_counter'] - df['fifo_full_ticks_count']   

    return df

def postprocess_df_metrics(df, columns_order=DataFrame_Columns_Order.data_pkt_columns):
    ''' This function can only be called after the whole dataframe was collected. '''
    
    if df.empty:
        print("WARNING: Postprocessing can't be done because df is empty, returning empty df.")
        return df
    
    for col in ['clk_counter', 'fifo_full_ticks_count', 'clk_counter_halt_agnostic']:
        if col in df.columns:
            df.loc[0, col] = 0
            df[f'total_{col}'] = df[col].cumsum()     
    df = reorder_df_columns(df, columns_order=columns_order)
    return df

input_buffer_all_transfers_copied = []


def collect_program_data(input_buffer, dont_decode=False, dont_wait=False, dont_parse=False, copy_collected=False, 
                         execution_time_limit=None, packet_format=Packet_Format.data_pkt, 
                         columns_order=DataFrame_Columns_Order.data_pkt_columns, debug=False):
    global dma_rec, input_buffer_all_transfers_copied, BUFFER_ITEM_CAPACITY
    
    if (execution_time_limit is not None) and dont_parse:
        raise Exception("Execution time can't be checked without parsing received data")

    i = 0
    total_items = 0
    input_buffer_all_transfers_copied = []
    df_metrics = pd.DataFrame()
    total_execution_clocks = 0
    while True:
        # transfer all collected data
        if debug:
            print(f'Initiating DMA transfer i={i}')
        items_transferred = get_dma_transfer(input_buffer, dma_rec, timeout_ms=1000)#, dont_wait=True) 
        if items_transferred < 1:
            if debug:
                print("NO ITEMS TRANSFERRED, PROGRAM LIKELY FINISHED")
            break
        i += 1
        
        total_items += items_transferred
        if dont_parse:
            if debug:
                print(f'Transfer {i} finished (not parsing), items_transferred={items_transferred}.')
            if copy_collected:
                if debug:
                    print(f'Copying buffer to input_buffer_all_transfers_copied')
                
                input_buffer_all_transfers_copied.append( 
                    input_buffer[:int(BUFFER_ITEM_CAPACITY * INPUT_BUFFER_LOCATIONS_PER_ITEM+1)].copy() )
                if debug:
                    print(f'Buffer was copied')
            
#             # TODO: FIX, THIS IS GOING TO CAUSE PROBLEMS
#             if (items_transferred) != TLAST_INTERVAL:
#                 print(f'All DMA transfers completed (no parsing), total_items={total_items}. It is assumed that all transfers completed because items_transferred ({items_transferred}) != TLAST_INTERVAL ({TLAST_INTERVAL}).')
#                 return None
            continue 
            
        if debug:
            print(f'Transfer {i} finished, items_transferred={items_transferred}, parsing...')
        df_metrics_single = parse_input_buffer(input_buffer, items_transferred, packet_format=packet_format)
        df_metrics_single = preprocess_df_metrics(df_metrics_single, dont_decode=dont_decode)
        df_metrics = df_metrics.append(df_metrics_single, ignore_index=True)
        total_execution_clocks += df_metrics_single['clk_counter_halt_agnostic'].sum()

#         if df_metrics_single['instr_names'][-1].lower() == 'wfi':
#             break
            
        execution_time_ms = (total_execution_clocks / CLK_SPEED * 1000)
        if execution_time_limit is not None and execution_time_limit < execution_time_ms:
            print(f'Execution time limit ({execution_time_limit}ms) was reached, tracing is stopped. (execution time={execution_time_ms}ms)')
            break
        if debug:
            print(f'execution_time_ms = {execution_time_ms}')
            
    if debug:
        print(f'All DMA transfers completed, total_items={total_items}.') 
    df_metrics = postprocess_df_metrics(df_metrics, columns_order=columns_order)
    return df_metrics

def run_and_collect(stdin, input_buffer=input_buffer, dont_decode=False, dont_parse=False, copy_collected=False, 
                    execution_time_limit=None, packet_format=Packet_Format.data_pkt, 
                    columns_order=DataFrame_Columns_Order.data_pkt_columns, 
                    debug=False):
    ''' dont_decode=True saves time (otherwise instruction assembly string is created from hex instruction value) '''
    # set CPU into inactive state (active-low reset is set LOW)
    gpio_rst_n_out.write(0)
    # activate continous_monitoring_system in case if it's stopped by previously 
    # encountered "wait for interrupt" (WFI) instruction
    cms_ctrl.reset_wfi_wait()
    # send standard input into a buffer, this way it will be ready
    # immediately after CPU starts running the program    
    console_io.send(stdin, end_byte=ord('\n')) # '\n' is hardcoded here specifically for "stack-mission.c" program
    
    # get transfer and ignore it just in case if internal trace storage is not empty
    items_transferred = get_dma_transfer(input_buffer, dma_rec)
    
    reset_cpu()
    df = collect_program_data(input_buffer, dont_decode=dont_decode, dont_wait=False, dont_parse=dont_parse, 
                              copy_collected=copy_collected, execution_time_limit=execution_time_limit, 
                              packet_format=packet_format, columns_order=columns_order, debug=debug)
    stdout = console_io.read()
    return df, stdout
    
def get_performance_stats(df, clk_speed=50_000_000):
    halted_time = df['fifo_full_ticks_counts'][1:].sum() / clk_speed
    normal_run_time = df['clk_counter'][1:].sum() / clk_speed - halted_time
    performance_decrease = 100.0 - normal_run_time / (normal_run_time + halted_time) * 100
    return halted_time, normal_run_time, performance_decrease

def print_performance_stats(df):
    halted_time, normal_run_time, performance_decrease = get_performance_stats(df)
    print(f'normal_run_time = {normal_run_time}s')
    print(f'halted_time = {halted_time}s')
    print(f'performance_decrease = {performance_decrease}%')

    
def get_available_disk_space(human_readable=False):
    if human_readable:
        return os.popen('df -h --output=avail,source | grep root').read().strip().split(' ')[0]
    return int(os.popen('df --output=avail,source | grep root | cut -d " " -f 1').read().strip())

def reorder_df_columns(df, columns_order):
    ''' function to reorder specified columns, columns not mentioned
    in columns_order will be appended to the end of the dataframe '''
    columns = df.columns.tolist()
    for column in columns_order:
        assert column in columns, f'ERROR: reorder_df_columns "{column}" column was not found in columns (columns={columns})'
        columns.remove(column)
    columns = columns_order + columns
    return df[columns]

event_names = read_performance_event_names(PERFORMANCE_EVENTS_FNAME)

# mem = !cat /proc/meminfo | grep 'MemFree'
# print(mem)

# set processor into reset state
gpio_rst_n_out.write(0)

# load the default program
#bram_loader.start_load(PROGRAMS_DIR / 'ECG/ecg.bin')
# bram_loader.start_load(PROGRAMS_DIR / 'HW/hello_world.bin')
# while not bram_loader.finished_loading():
#     print(bram_loader.get_load_progress())
#     time.sleep(1)

print()
print('Initialization done')
print()
print(operational_config)


/usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/overlay.py:681: UserWarning: Interrupt s2mm_introut not created: Could not find UIO device for interrupt pin for IRQ number 62
  warnings.warn('Interrupt {} not created: {}'.format(


continuous_monitoring_system_controller config loaded:
{'atf_active': True,
 'atf_mode': 1,
 'basic_trace_filter_mode': 0,
 'basic_trace_filter_time_interval_ticks': 10,
 'basic_trace_filter_time_interval_type': 1,
 'external_trace_filter_mode_enabled': False,
 'feature_extractor_halting_cpu_enabled': False,
 'halting_cpu': True,
 'monitored_address_range_lower_bound': 4095,
 'monitored_address_range_lower_bound_enabled': False,
 'monitored_address_range_upper_bound': 2147483903,
 'monitored_address_range_upper_bound_enabled': False,
 'tlast_interval': 0,
 'trigger_trace_end_address': 2147483910,
 'trigger_trace_end_address_enabled': False,
 'trigger_trace_start_address': 4096,
 'trigger_trace_start_address_enabled': False}
buffer_length = 6250000
Performance events count = 8

Initialization done

Operational_Config({'disable_saving': False,
 'f_name': 'config.pickle',
 'items_collected_processing_limit': 500,
 'lack_of_matches_threshold_multiplier_of_max_interval': 1.5,
 'periodic_sen

In [2]:
tcp_server.set_verbose(True)

# Configuration of CMS

Settings of continuous monitoring system like:
* monitored range (optional)
* monitoring start trigger address (optional)
* monitoring end trigger address (optional)
* auto-halting cpu when internal trace storage is full (optional)
* trace filtering method (basic, watchpoint-based, external-pin triggered)
* set watchpoints conditions upon which data is collected

In [3]:
def setup_cms(cms_ctrl):
    
    cms_ctrl.disable_storing_config() 
    # Triggerring (exact address must match to start/stop trace)
    cms_ctrl.set_trigger_trace_start_address(0x1000)
    cms_ctrl.set_trigger_trace_end_address(0x80000106)  
    cms_ctrl.set_trigger_trace_start_address_enabled(False)
    cms_ctrl.set_trigger_trace_end_address_enabled(False)

    # Filtering (any address between lower bound and upper bound will be collected)
    cms_ctrl.set_monitored_address_range_lower_bound(0x0FFF)     #(0x80000000)
    cms_ctrl.set_monitored_address_range_upper_bound(0x800000FF)
    cms_ctrl.set_monitored_address_range_lower_bound_enabled(False)
    cms_ctrl.set_monitored_address_range_upper_bound_enabled(False)
    
    # Allow further trace collection if last traced program used "wfi"
    # (wait for interrupt) instruction which stops the trace.
    cms_ctrl.reset_wfi_wait()
    cms_ctrl.set_tlast_interval(TLAST_INTERVAL)
    
    
    # CPU HALTING IS DISABLED TO PREVENT HALT ON TOO FREQUENT WATCHPOINT MATCHES
    # As described in: https://trello.com/c/6eHG0Siu/17-prevent-cpu-halt-on-too-frequent-watchpoints
    # IF THE FIFO IS FULL, NOTIFY THE GUI THROUGH "add_points" MESSAGE OR ANY OTHER 
    # SPECIAL MESSAGE
    
#      cms_ctrl.enable_halting_cpu()
    cms_ctrl.disable_halting_cpu()
    
    cms_ctrl.reset_atf()
    
    # 3 MAIN TRACE FILTER OPTIONS:
    # - basic
    # - advanced (watchpoint-based)
    # - external (data is collected when external input to CMS is high)
       
    # Basic trace filter configuration (affects atf mode too)
#     cms_ctrl.set_basic_trace_filter_mode_jump_branch_return()
    cms_ctrl.set_basic_trace_filter_mode_all_instructions()
    # cms_ctrl.set_basic_trace_filter_mode_time_interval()
    # cms_ctrl.set_basic_trace_filter_time_interval_ticks(1)
    
    #cms_ctrl.set_basic_trace_filter_mode(BASIC_TRACE_FILTER_MODE.DISABLED)
    
    # Advanced trace filter (ATF) configuration
    # DIRECT MATCH ATF WATCHPOINTS (determining when data is collected):
    #cms_ctrl.set_atf_match_watchpoint(0, {'pc':0x8000076c})
    #cms_ctrl.set_atf_match_watchpoint(0, {'pc':0x80000760}) # ecg_baseline wait_ms
    #cms_ctrl.set_atf_match_watchpoint(0, {'pc':0x800008B0}) # ecg_baseline wait_ms_2
    cms_ctrl.set_atf_mode(ATF_MODE.ANOMALY_DETECTION) # alternative: ATF_MODE.PATTERN_COLLECTION
    cms_ctrl.enable_atf()
    #cms_ctrl.disable_atf() 
    
    cms_ctrl.disable_external_trace_filter() 
    
    #cms_ctrl.enable_feature_extractor_halting_cpu()
    cms_ctrl.disable_feature_extractor_halting_cpu()
    
    cms_ctrl.enable_storing_config()
    cms_ctrl.store_config()


def set_watchpoint_conditions(conditions):
    cms_ctrl.reset_atf()
    for i, wp in enumerate(conditions): # make sure the number of chosen conditions is not too many (max 8 as of 12/12/2023) 
        cms_ctrl.set_atf_match_rule(i, values_dict={}, seed=wp['seed'], mask=wp['mask'], bits_to_use=wp['bits_to_use'])

def setup_cms_for_testing_feature_extractor():
    cms_ctrl.set_basic_trace_filter_mode(BASIC_TRACE_FILTER_MODE.DISABLED)
    cms_ctrl.disable_atf()
    # "valid" output of feature extractor is the external_trace_filter_keep_item signal of CMS
    cms_ctrl.enable_external_trace_filter() 
    cms_ctrl.enable_feature_extractor_halting_cpu()
    cms_ctrl.enable_halting_cpu()

    
# setup_cms(cms_ctrl)

# no need to setup_cms anymore because it's loaded/stored
cms_ctrl.print_config()


Continuous Monitoring System Controller configuration:
    trigger_trace_start_address_enabled: False
    trigger_trace_end_address_enabled: False
    trigger_trace_start_address: 0x1000
    trigger_trace_end_address: 0x80000106
    monitored_address_range_lower_bound_enabled: False
    monitored_address_range_upper_bound_enabled: False
    monitored_address_range_lower_bound: 0xfff
    monitored_address_range_upper_bound: 0x800000ff
    basic_trace_filter_mode: JUMP_BRANCH_RETURN
    basic_trace_filter_time_interval_ticks: 10
    basic_trace_filter_time_interval_type: 1
    external_trace_filter_mode_enabled: False
    feature_extractor_halting_cpu_enabled: False
    atf_mode: ANOMALY_DETECTION
    atf_active: True
    tlast_interval: 0
    halting_cpu: True



# TCP server setup
Functions that start with "rpc_" can be called from the Esp32 (with a display).

In [4]:
# # Initially it will be 1, during the calibration phase it will
# # be automatically adjusted to lower value (while performing
# # the same actions as during training).
# similarity_threshold = 1.0

# If items collected from fifo is above ITEMS_COLLECTED_PROCESSING_LIMIT, items will not be processed
# and data loss will be indicated to GUI controller
#
# If it happens during training, it means that training results are invalid and 
# training should be done with less frequently matching watchpoints.
# 
# If it happens during testing, it should be treated as anomaly.
# (anomalies in the number of collected items also could be detected,
#  but that's another story)
# (limit must be not larger than internal FIFO size)
# ITEMS_COLLECTED_PROCESSING_LIMIT = 1000
# EDIT: it got replaced with operational_config.get_items_collected_processing_limit()

# setup database with allowed barcodes
db_barcodes = SQL_Barcodes_DB()
db_barcodes.reset_table()
db_barcodes.init_with_default_barcodes()

# lock to protect is_running variable (set in rpc and read in "operate" thread)
# but also other variables in future if needed
server_data_lock = Lock()

class MODE:
    ''' Operational mode, controlled by TCP client.
    This isn't any internal hardware mode, it is just for this PYNQ script, 
    and to allow the TCP client to control what should happen. '''
    # these can be used with bitwise operators (need to be careful if new modes are added)
    IDLE = 0
    TRAINING = 0b1
    TESTING = 0b10
    TRAINING_AND_TESTING = 0b11
    
    # This mode should be done just after training.
    # While in this mode, we should repeat all the 
    # same actions that were done during training
    # but this time the lowest encountered similarity
    # will become detection threshold for the testing
    # mode.
    DETECTION_THRESHOLD_CALIBRATION = 0b100 

def remove_too_varying_performance_events(events):
    events_copy = list(events)
    to_remove = ['Core__1_BUSY_NO_CONSUME']
    for ev in to_remove:
        index = event_names.index(ev)
        del events_copy[index]
    return events_copy

# used_events = remove_too_varying_performance_events(event_names)
used_events = event_names

USED_PERFORMANCE_EVENTS_COUNT = len(used_events)
    
# declaration of some variables that are controlled by the client.
mode = MODE.IDLE    

anomaly_detection = Anomaly_Detection() # model for anomaly detection
# Using line below splits datasets into many subdatasets
# grouped together by unique program counter values.
def set_special_columns_indices():
    global anomaly_detection
    enabled_features = [name for name,is_enabled in anomaly_detection.get_used_features().items() if is_enabled]
    anomaly_detection.set_special_columns_indices([enabled_features.index('pc')])
set_special_columns_indices()
    
is_arbitrary_halt_active = False
loaded_program = 'None'
is_running = False

# This multiplier is used for notifying GUI about lack of watchpoint matches.
# If there is no data collected within 1.5 * max_recorded_interval_during_training
# then lack of matches anomaly is sent.
# TODO: must be adjustable and persistent
# lack_of_matches_threshold_multiplier_of_max_interval = 1.5
# EDIT: it got replaced by operational_config.get_lack_of_matches_threshold_multiplier_of_max_interval()

from advanced_trace_filter import ATF_Watchpoints
atf_watchpoints = ATF_Watchpoints(cms_ctrl)
atf_watchpoints.load_watchpoints()
atf_watchpoints.push_all_watchpoints_to_cms()

def list_subfolders_with_paths(path):
    ''' From: https://stackoverflow.com/a/59938961/4620679 '''
    return [f.path for f in os.scandir(path) if f.is_dir()]


def generate_status_update_dict():
    global anomaly_detection, mode, is_arbitrary_halt_active, loaded_program, is_running, atf_watchpoints
    global pynq_restarted, bram_loader, atf_watchpoints, cms_ctrl
    # function created to create consistent message for "status_update"
    # and return of "rpc_update_status", so both can be parsed
    # using the same routine
    return {
        'pynq_restarted' : pynq_restarted,
        'dataset_size' : anomaly_detection.get_dataset_size(),
        'mode' : mode,
        'is_halted' : is_arbitrary_halt_active,
        'loaded_program' : loaded_program,
        'is_running': is_running,
        'program_load_progress' : bram_loader.get_load_progress(),
        'similarity_threshold' : anomaly_detection.get_similarity_threshold(),
        #'features_keys' : Packet_Format.get_anomaly_detection_features_names(),
        'atf_watchpoints' : atf_watchpoints.get_watchpoints_as_strings(),
        'cms_ctrl_config' : cms_ctrl.get_config(),
        
        # model related
        'model_has_unsaved_changes' : anomaly_detection.has_unsaved_changes(),
        'model_name_current' : anomaly_detection.get_current_model_name(),
        'model_max_interval' : float(anomaly_detection.get_max_interval()),
        'model_used_features' : anomaly_detection.get_used_features(),
        
        # operational config
        'operational_config' : operational_config.get_config()
    }

#############################################################################
# API calls for the TCP server (that TCP clients may call whenever they want)
def rpc_list_programs():
    ''' TCP server API.'''
    # key=main program name (dir name in programs) value=list of programs (e.g. ecg_baseline.bin, ecg_ino_leak.bin)
    programs = {}
    for path in list_subfolders_with_paths(str(PROGRAMS_DIR)):
        p_name = os.path.basename(path)
        programs[p_name] = sorted([f_name.split('.')[0] for f_name in os.listdir(path) if f_name.endswith(".bin")])
    return programs
    #response = {'programs':programs}
    #return json.dumps(response)

def rpc_list_objdumps():
    objdumps = {}
    for path in list_subfolders_with_paths(str(PROGRAMS_DIR)):
        p_name = os.path.basename(path)
        objdump_path = Path(path) / 'objdump'
        objdumps[p_name] = sorted([f_name.split('.')[0] for f_name in os.listdir(objdump_path) if f_name.endswith(".dump")])
    return objdumps

def rpc_get_objdump_data(category, objdump_fname):
    # {'_start': {'80000000': {'name': 'entry', 'type': 'entry'},
    #             '80000004': {'branch_destination': '<park>',
    #                          'name': 'BNEZ',
    #                          'type': 'branch'},
    #             '80000010': {'branch_destination': '<main>',
    #                          'name': 'J',
    #                          'type': 'branch'}},
    # 'main': {'80000038': {'name': 'entry', 'type': 'entry'},
    #           '80000088': {'branch_destination': '<main+0x6c>', 'name': 'BEQZ', 'type': 'branch'},
    #           '80000094': {'name': 'uart_gpio_puts', 'type': 'function'},    
    if not objdump_fname.endswith('.dump'):
        objdump_fname += '.dump'
        
    full_fname = PROGRAMS_DIR / Path(category) / f'objdump/{objdump_fname}'
    try:
        return parse_objdump(full_fname)
    except Exception as e:
        error_msg = f'ERROR: failed parsing "{full_fname}" file: ' + str(e)
        print(error_msg)
        return error_msg

def rpc_load_program(name):
    ''' TCP server API. '''
    global is_arbitrary_halt_active, loaded_program
    if not name.endswith('.bin'):
        name += '.bin'
    for dirpath, d_names, f_names in os.walk(str(PROGRAMS_DIR)):
        for f_name in f_names:
            if f_name != name:
                continue
            full_path = os.path.join(dirpath, name)
            gpio_rst_n_out.write(0)
            bram_loader.start_load(full_path)
            while not bram_loader.finished_loading():
                send_file_load_progress(bram_loader.get_load_progress())
                time.sleep(0.5)
            send_file_load_progress(bram_loader.get_load_progress())
            
            if is_arbitrary_halt_active:
                cms_ctrl.deactivate_arbitrary_halt()
                is_arbitrary_halt_active = False
            loaded_program = name.split('.')[0]
            return f"OK: loaded {name} program."
            #return json.dumps({'status_update': f'OK: ran {name} program'})
    return f"ERROR: didn't find {name} program"

def rpc_run():
    ''' TCP server API. '''
    global is_arbitrary_halt_active, is_running, server_data_lock
    if not is_arbitrary_halt_active:
        reset_cpu()
    else:
        cms_ctrl.deactivate_arbitrary_halt()
        is_arbitrary_halt_active = False
    with server_data_lock:
        is_running = True
    return "OK"

def rpc_halt():
    ''' TCP server API. '''
    global is_arbitrary_halt_active, is_running, server_data_lock
    if is_arbitrary_halt_active:
        return 'Program was halted anyway'
    is_arbitrary_halt_active = True
    cms_ctrl.activate_arbitrary_halt()
    with server_data_lock:
        is_running = False
    return 'CPU halted'
    
def rpc_enable_training():
    global mode
    mode |= MODE.TRAINING
    return mode
    
def rpc_disable_training():
    global mode
    mode &= ~MODE.TRAINING
    return mode

def rpc_enable_testing():
    global mode
    mode |= MODE.TESTING
    return mode

def rpc_disable_testing():
    global mode
    mode &= ~MODE.TESTING
    return mode

def rpc_enable_detection_threshold_calibration():
    global mode
    mode |= MODE.DETECTION_THRESHOLD_CALIBRATION
    return mode
    
def rpc_disable_detection_threshold_calibration():
    global mode
    mode &= ~MODE.DETECTION_THRESHOLD_CALIBRATION
    return mode

def rpc_reset_dataset():
    global anomaly_detection
    anomaly_detection.reset_dataset()
    return 'Dataset resetted'

def rpc_update_status():
    return generate_status_update_dict()
#             {'dataset_size' : anomaly_detection.get_dataset_size(),
#             'mode' : mode,
#             'is_halted' : is_arbitrary_halt_active,
#             'loaded_program' : loaded_program,
#             'is_running': is_running,
#             'atf_watchpoints' : atf_watchpoints.get_watchpoints_as_strings()}

def rpc_set_atf_watchpoint(index, is_active, json_str_attributes_dict, json_str_attributes_notes_dict={}):
    global atf_watchpoints
    if type(is_active) == str:
        is_active = (is_active.lower() == 'true' or is_active == '1')
    index = int(index)
        
    try:
        attributes_dict = json.loads(json_str_attributes_dict)
    except Exception as e:
        error_msg = 'ERROR: rpc_set_atf_watchpoint: ' + str(e)
        print(error_msg)
        return error_msg
    try:
        attributes_notes_dict = json.loads(json_str_attributes_notes_dict)
    except Exception as e:
        error_msg = 'ERROR: rpc_set_atf_watchpoint (setting attributes_notes_dict): ' + str(e)
        print(error_msg)
        attributes_notes_dict = {}
    atf_watchpoints.set_watchpoint(index, attributes_dict, is_active, attributes_notes_dict=attributes_notes_dict)
    return f"OK_{index}"

def rpc_atf_watchpoint_set_active(index, state):
    global atf_watchpoints
    print(f'rpc_atf_watchpoint_set_active index={index} state={state}')
    
    if type(is_active) == str:
        is_active = (is_active.lower() == 'true' or is_active == '1')
    index = int(index)
    success = atf_watchpoints.set_watchpoint_active(index, state)
    return "OK" if success else f"WARNING: Watchpoint with index={index} wasn't there"

def rpc_remove_atf_watchpoint(index):
    global atf_watchpoints
    index = int(index)
    success = atf_watchpoints.remove_watchpoint(index)
    return "OK" if success else f"WARNING: Watchpoint with index={index} wasn't there"

def rpc_set_similarity_threshold(threshold):
    global anomaly_detection
    anomaly_detection.set_similarity_threshold(threshold)
    return anomaly_detection.get_similarity_threshold()

def rpc_list_available_models():
    return anomaly_detection.list_datasets()

def rpc_save_detection_model(name):
    anomaly_detection.store_dataset(name)
    return "OK"
    
def rpc_load_detection_model(name):
    global anomaly_detection, used_features
    anomaly_detection.load_dataset(name)
    used_features = anomaly_detection.get_used_features()
    return "OK"

def rpc_send_stdin(stdin_str):
    console_io.send(stdin_str)
    return "OK"

def rpc_read_stdout():
    # if this is going to be implemented for some reason (e.g. viewing stdout in GUI)
    # then stdin_stdout_communication() function will need to store received stdout
    # in some global variable or some object
    return console_io.read()

def rpc_readlines_stdout():
    return console_io.read().split('\n')

def rpc_set_cms_ctrl_attributes(json_str_attributes_dict):
    try:
        attributes_dict = json.loads(json_str_attributes_dict)
    except Exception as e:
        error_msg = 'ERROR: rpc_set_cms_ctrl_attributes: ' + str(e)
        print(error_msg)
        return error_msg
    try:
        errors = cms_ctrl.update_attributes(attributes_dict)
    except Exception as e:
        error_msg = 'ERROR: rpc_set_cms_ctrl_attributes: ' + str(e)
        print(error_msg)
        return error_msg
    return 'OK' if not errors else errors

def rpc_set_model_max_interval(interval):
    global anomaly_detection
    anomaly_detection.set_max_interval(interval)
    return 'OK'

def rpc_set_operational_config(json_str_attributes_dict):
    try:
        attributes_dict = json.loads(json_str_attributes_dict)
    except Exception as e:
        error_msg = 'ERROR: rpc_set_operational_config: ' + str(e)
        print(error_msg)
        return error_msg
    try:
        errors = operational_config.update_attributes(attributes_dict)
    except Exception as e:
        error_msg = 'ERROR: rpc_set_operational_config: ' + str(e)
        print(error_msg)
        return error_msg
    return 'OK' if not errors else errors

def rpc_set_feature_enabled(feature_name, is_enabled):
    if type(is_enabled) == str:
        is_enabled = is_enabled.lower() == 'true'
    error_msg = anomaly_detection.set_feature_enabled(feature_name, is_enabled)
    if error_msg:
        print(error_msg)
    # update special column indices in anomaly_detection
    set_special_columns_indices()
    return 'OK' if not error_msg else error_msg

# def rpc_set_periodic_send_interval_seconds(val):
#     global operational_mode
#     operational_mode.set_periodic_send_interval_seconds()
#     return 'OK'

# def rpc_set_items_collected_processing_limit(val):
#     global operational_mode
#     operational_mode.set_items_collected_processing_limit(val)
#     return 'OK'

# def rpc_set_lack_of_matches_threshold_multiplier_of_max_interval(val):
#     global operational_mode
#     operational_mode.set_lack_of_matches_threshold_multiplier_of_max_interval(val)
#     return 'OK'
    
#############################################################################
# Functions that the PYNQ board can use to notify all clients about 

def send_sensors_data(df_sensors, sensors_to_send):
    msg_to_server = ''
    for i in range(df_sensors.shape[0]):
        for col in sensors_to_send: #df_sensors.columns:
            val = float(df_sensors[col].iloc[i]) / 60000.0
            msg_to_server += f'add_point:{col},{val}\n'
    #print(msg_to_server)
    tcp_server.send_to_all(msg_to_server) 

def send_file_load_progress(percent):
    tcp_server.send_to_all(json.dumps({
        'status_update' : {
            'program_load_progress' : percent
        }
    }))

def send_similarity_threshold(threshold):
    tcp_server.send_to_all(json.dumps({
        'status_update' : {
            'similarity_threshold' : float(threshold)
        }
    }))

def send_model_unsaved_changes(state):
    tcp_server.send_to_all(json.dumps({
        'status_update' : {
            'model_has_unsaved_changes' : state
        }
    }))

def send_model_current_name(name):
    tcp_server.send_to_all(json.dumps({
        'status_update' : {
            'model_name_current' : name
        }
    }))

def send_model_max_interval(interval):
    tcp_server.send_to_all(json.dumps({
        'status_update' : {
            'model_max_interval' : float(interval)
        }
    }))

def send_stdout(msg):
    if not msg: return
    tcp_server.send_to_all(json.dumps({
        'stdout' : msg
    }))

def send_stdin(msg):
    if not msg: return
    tcp_server.send_to_all(json.dumps({
        'stdin' : msg
    }))

# PERIODIC_SEND_INTERVAL = 0.8 # in seconds 
# EDIT: replaced by operational_config.get_periodic_send_interval_seconds()

def send_periodic_update(similarities, items_since_last_send, clk_time_since_last_send, halted_time_since_last_send, 
                         mode, send_dataset_size=False, processing_limit_exceeded=False, lack_of_matches=False):
    global anomaly_detection, operational_config
    
    if mode & MODE.TESTING:
        number_of_anomalies = sum(1 for s in similarities if s < anomaly_detection.get_similarity_threshold())
    else:
        number_of_anomalies = 0    
        
    
    if processing_limit_exceeded or lack_of_matches:
        # processing was not done in this case and all collected items were ignored
        avg_sim_bot_1 = 0.0
        avg_sim = 0.0
        performance_rate = 1.0 
    else:
        avg_sim_bot_1 = 1 if not similarities else np.mean( sorted(similarities)[:math.ceil(len(similarities)/100)] )
        avg_sim = 1 if not similarities else np.mean(similarities)
    #     total_exec_time = clk_time_since_last_send + halted_time_since_last_send
    #     print('total_exec_time =', total_exec_time)
    #     print('clk_time_since_last_send =', clk_time_since_last_send)
    #     print('halted_time_since_last_send =', halted_time_since_last_send)
        performance_rate = (1 - halted_time_since_last_send / (clk_time_since_last_send or 1)) # "or 1" prevents division by 0
        
    dict_ = {
        'add_points' : {
            'Perf' : [performance_rate],
            'Avg sim' : [avg_sim],
            'Avg sim bot-1%' : [avg_sim_bot_1],
            'Items collected' : [items_since_last_send],
            'Anomalies' : [number_of_anomalies],
            'similarity_threshold' : [anomaly_detection.get_similarity_threshold()],
            'data_loss' : [1.0 if processing_limit_exceeded else 0.0],
            'data_loss_2' : [1.0 if processing_limit_exceeded else 0.0], # for displaying on 2nd graph
            'Items collected processing limit' : [float(operational_config.get_items_collected_processing_limit())],
            'lack_of_matches' : [1.0 if lack_of_matches else 0.0],
            'lack_of_matches_2' : [1.0 if lack_of_matches else 0.0]  # for displaying on 2nd graph
        }
    }
    if send_dataset_size:
        size = anomaly_detection.get_dataset_size()
        dict_['status_update'] : {'dataset_size' : size}
        dict_['add_points']['dataset_size'] = [size]
    
    tcp_server.send_to_all(
        json.dumps(dict_)
    )

def send_new_anomaly(metrics_dict, similarity, features_vector, most_similar_vector, 
                     data_loss=False, lack_of_matches=False):  
    if not data_loss and not lack_of_matches:
        halt_agnostic_clk_counter = metrics_dict['clk_counter'] - metrics_dict['fifo_full_ticks_count']
        if most_similar_vector is None:
            # most_similar_vector can be None if the PC of features_vector isn't found 
            # at all in the dataset (because dataset is split into subdatasets grouped
            # by PC), in that case all "-1s" are sent 
            most_similar_vector = np.full_like(features_vector, -1.0)

        pc = f"0x{metrics_dict['pc']:X}"
    else:
        # if too many watchpoints matched (and processing wasn't done)
        features_count = len(Packet_Format.get_anomaly_detection_features_names())
        pc = "-"
        halt_agnostic_clk_counter = '-'
        similarity = 0
        features_vector = [-1] * features_count
        most_similar_vector = features_vector
        
        
    tcp_server.send_to_all(json.dumps({
        'new_anomaly' : {
            'pc' : pc,                                  # string
            'time' : datetime.datetime.now().strftime("%H:%M:%S"),               # string
            'total_clk_counter' : str(halt_agnostic_clk_counter),                # string
            'similarity' : similarity,                                           # float between 0 and 1
            'features_vector' : list(float(v) for v in features_vector),         # list of floats
            'most_similar_vector' : list(float(v) for v in most_similar_vector),  # list of floats
            'data_loss' : data_loss,
            'lack_of_matches' : lack_of_matches
        }
    }))
    
def send_raw_data(raw_data_dict):
    try:
        tcp_server.send_to_all(json.dumps({
            'raw_data_dict' : raw_data_dict
        }))
    except Exception as e:
        print(f"ERROR send_raw_data: {e}")
        print(raw_data_dict)

def send_used_features():
    global anomaly_detection
    tcp_server.send_to_all(json.dumps({
        'status_update' : {
            'model_used_features' : anomaly_detection.get_used_features()
        }
    }))

# all functions from this file that start with "rpc_"
all_rpcs = [func for name,func in inspect.getmembers(sys.modules[__name__]) if (inspect.isfunction(func) and name.startswith('rpc_'))]
tcp_server.register_rpcs(all_rpcs)

tcp_server.start()
print(f'TCP server can be accessed at: {get_my_ip()}:{TCP_SERVER_PORT}')

tcp_server.send_to_all(json.dumps({'status_update': generate_status_update_dict()}))
pynq_restarted = False

TCP server can be accessed at: 192.168.0.111:9093
Server -> Client:
{'status_update': {'atf_watchpoints': {'0': {'active': True,
                                             'attributes': {'pc': '800000fc'},
                                             'attributes_notes': {'pc': 'getchar'}},
                                       '1': {'active': False,
                                             'attributes': {'pc': '800001f4'},
                                             'attributes_notes': {'pc': 'is_barcode_valid'}},
                                       '2': {'active': False,
                                             'attributes': {'pc': '800002c0'},
                                             'attributes_notes': {'pc': 'wait_ms+0x24'}}},
                   'cms_ctrl_config': {'atf_active': True,
                                       'atf_mode': 1,
                                       'basic_trace_filter_mode': 0,
                                       'basic_trace_filter

In [5]:
tcp_server.set_verbose(False)

# Main operation
Code part below should run a loop that will collect program metrics and depending on the state set by the TCP client:
- update model with training data
- calculate similarity to trained model and update client about it
- update client with metrics collection status


Similarity data may sent to client may be:
- the number of anomalous items collected (e.g. with similarity < 0.9)
- average similarity since last update (not sure, this may just be a distraction from the number of anomalies which is the most important)


Metrics collection status may include:
- the number of collected items since last update
- performance rate (total_execution_time - halted_time / total_execution_time)

In [6]:



chunks_per_item = math.ceil(AXI_DATA_WIDTH/64)
# CLK_LOCATION = PERFORMANCE_EVENTS_COUNT * PERFORMANCE_COUNTER_WIDTH + PERFORMANCE_COUNTERS_OVERFLOW_MAP_WIDTH + PC_WIDTH
# HALTED_CLK_LOCATION = CLK_LOCATION + CLK_COUNTER_WIDTH + INSTR_WIDTH

# variables below allow to interact with the thread running "operate" function
end_operate_thread = False
print_stats = False

#mode = MODE.TRAINING
#mode = MODE.TESTING
#mode = MODE.TRAINING_AND_TESTING

similarities = []

console_io.read() # ignore all previous stdout

console_io.set_on_send_callback(lambda msg: send_stdin(msg))
console_io.set_on_receive_callback(lambda msg: send_stdout(msg))

anomaly_detection.set_on_has_unsaved_changes_callback(lambda state: send_model_unsaved_changes(state))
anomaly_detection.set_on_model_current_name_callback(lambda name: send_model_current_name(name))
anomaly_detection.set_on_max_interval_change_callback(lambda interval: send_model_max_interval(interval))
anomaly_detection.set_on_feature_enabled_change_callback(lambda name, state: send_used_features())

not_processed_stdin = ""
def stdin_stdout_communication():
    global not_processed_stdin
    try:
        stdout_whole_str = console_io.read()
        if not stdout_whole_str:
            return
#         send_stdout(stdout_whole_str)
        lines = stdout_whole_str.split('\n')
        if not lines:
            return
        # don't process stdin that does not end with '\n'
        if not_processed_stdin:
            lines[0] = not_processed_stdin + lines[0]
            not_processed_stdin = ""
        # if last line is not empty, it means that
        # '\n' wasn't at the end of the received text
        # so the last string should be saved for further processing
        if lines[-1] != "":
            not_processed_stdin = str(lines[-1])
        # either way the last line must be removed because either it's:
        # - empty (due to text ending with '\n')
        # - not complete message that should not be processed (due to lack of '\n' at the end)
        del lines[-1]
       
        for line in lines:
            if not line:
                continue
            if line.startswith("bc:"):
                bc = line.split("bc:")[1]
                is_in_db = 1 if db_barcodes.is_barcode_in_db(bc) else 0
                console_io.send(f'bc:{is_in_db}', end_byte=ord('\n'))
            print(line)
    except UnicodeDecodeError:
        print('WARNING: UnicodeDecodeError in stdin_stdout_communication')

def operate():
    global end_operate_thread, print_stats
    global anomaly_detection
    global server_data_lock
    global is_running
    global operational_config
    
    # items received through DMA
    items_transferred = 0
    total_clk_time = 0
    total_halted_time = 0
    
    # metrics for client (display)
    items_since_last_send = 0
    clk_time_since_last_send = 0
    halted_time_since_last_send = 0
    last_send_time = time.time()
    processing_limit_exceeded = False
    # to prevent spamming send_new_anomaly
    processing_limit_exceeded_reseted = True
    
    # to recognize lack of watchpoint matches
    last_watchpoint_match_time = 0
    lack_of_matches = False
    # to prevent spamming send_new_anomaly
    lack_of_matches_reset = True
    
    while True:        
        if end_operate_thread:
            print('Exiting thread')
            return
        stdin_stdout_communication()
        
        if time.time() - last_send_time > operational_config.get_periodic_send_interval_seconds() and mode != MODE.IDLE:
            # send dataset_size update only if training or calibration is enabled
            send_dataset_size = (mode & MODE.TRAINING) or (mode & MODE.DETECTION_THRESHOLD_CALIBRATION)
            send_periodic_update(similarities, items_since_last_send, clk_time_since_last_send, 
                                 halted_time_since_last_send, mode, send_dataset_size=send_dataset_size,
                                 processing_limit_exceeded = processing_limit_exceeded,
                                 lack_of_matches = lack_of_matches)
            similarities.clear()
            items_since_last_send = 0
            clk_time_since_last_send = 0
            halted_time_since_last_send = 0
            last_send_time = time.time()
            
            processing_limit_exceeded = False

        with server_data_lock:
            is_running_copy = bool(is_running)
        if not is_running_copy or mode == MODE.IDLE:
            last_watchpoint_match_time = time.time()
            
#         # in case of training, setting it to 0 is ok
#         # in case of testing, setting it to 0 makes lack_of_matches
#         # not being set until at least 1 watchpoint is hit
#         if mode & MODE.TESTING:
#             last_watchpoint_match_time = time.time()
                
        items_transferred = get_dma_transfer(input_buffer, dma_rec)
        #events, events_overflows, pcs, clk_counters, instrs, instr_names, instr_strings, fifo_full_ticks_counts, all_gp_regs = parse_input_buffer(input_buffer, items_transferred, dont_decode=True)

        if not items_transferred:
            time.sleep(0.001)
            
            #if delta > anomaly_detection.get_lack_of_matches_threshold()
            if last_watchpoint_match_time and mode & MODE.TESTING:
                delta = time.time() - last_watchpoint_match_time
                lack_of_matches_threshold = anomaly_detection.get_max_interval() * operational_config.get_lack_of_matches_threshold_multiplier_of_max_interval()
                # if threshold was previously set and now exceeded
                if lack_of_matches_threshold and delta > lack_of_matches_threshold:
                    # if notification wasn't sent yet
                    lack_of_matches = True
                    if lack_of_matches_reset:
                        send_new_anomaly(None, None, None, None, lack_of_matches=lack_of_matches)
                        lack_of_matches_reset = False
                        
#             # don't count intervals between watchpoints when training/calibration is not enabled
#             if not mode & MODE.TRAINING and not mode & MODE.DETECTION_THRESHOLD_CALIBRATION:
#                 last_watchpoint_match_time = 0
            continue
        lack_of_matches = False
        lack_of_matches_reset = True
        
        if print_stats:
            print(items_transferred, end=', ')

        # recognize lack of matches
        if last_watchpoint_match_time and (mode & MODE.TRAINING or mode & MODE.DETECTION_THRESHOLD_CALIBRATION):
            delta = time.time() - last_watchpoint_match_time
            anomaly_detection.update_max_interval(delta)

        last_watchpoint_match_time = time.time()
        
        items_since_last_send += items_transferred
        
        # recognize too many matches (data_loss, because in that case collected items are not processed)
        processing_limit = operational_config.get_items_collected_processing_limit()
        if items_transferred > processing_limit:
            print(f'items_transferred ({items_transferred}) is above limit ({processing_limit}) ignoring chunk')
            processing_limit_exceeded = True
            # only consider data_loss an anomaly if it occurs during testing (called "monitoring" in GUI)
            if mode & MODE.TESTING:
                # this condition prevents spamming send_new_anomaly
                if processing_limit_exceeded_reseted:
                    send_new_anomaly(None, None, None, None, data_loss = True)
                    processing_limit_exceeded_reseted = False     
            # update last match time for the sake of recognizing lack of matches later
            continue
        
        processing_limit_exceeded_reseted = True
        

        
    #     events, events_overflows, pcs, clk_counters, instrs, instr_names, instr_strings, fifo_full_ticks_counts, all_gp_regs = parse_input_buffer(input_buffer, items_transferred, dont_decode=True)
    #     df = pd.DataFrame(zip(pcs,clk_counters,instrs,instr_names,instr_strings,fifo_full_ticks_counts), columns=['pc','clk_counter','instr', 'instr_names', 'instr_strings', 'fifo_full_ticks_counts'])
    #     # all_gp_regs is a list of dicts, it is joined below into the main dataframe
    #     df = df.join( pd.DataFrame.from_dict(all_gp_regs) )
    #     df.iloc[:,0] = df.iloc[:,0].apply(lambda x: f'{x:08X}')
    #     print( df.iloc[:items_transferred] )
        start = 0
        end = chunks_per_item
        processing_time_checkpoint = time.time()
        for i in range(items_transferred):
            
            fifo_item = int.from_bytes(bytes(input_buffer[start:end]), byteorder='little')            
    #         clk_count = (fifo_item >> CLK_LOCATION) & ((1 << 64)-1)
    #         halted_clk_count = (fifo_item >> HALTED_CLK_LOCATION) & ((1 << 64)-1)            
            metrics_dict = parse_fifo_item(fifo_item, Packet_Format.data_pkt)
#             perf_counters_values = list(Packet_Format.get_perf_counters_dict_from_metrics_dict(metrics_dict).values())
    
#             features_vector = Packet_Format.get_vector_for_anomaly_detection_from_metrics_dict(metrics_dict)
            
            used_features = anomaly_detection.get_used_features()
#             features_vector = [v for k,v in metrics_dict.items() if used_features.get(k, False)]
            # as of 24/03/24 it is important that the first feature is program counter
            # that is because in AnomalyDetection, the first feature is used to split model
            # into submodels (each for unique program counter value)
            features_vector = [metrics_dict.get(k) for k,v in used_features.items() if v == True]
            print(used_features)
            print(features_vector)
    
#             features_dict = Packet_Format.get_dict_for_anomaly_detection_from_metrics_dict(metrics_dict)
            
            # variable to avoid calculating the same thing twice
            if mode != MODE.IDLE:
                # specific_features = anomaly_detection.
                # Packet_Format.get_anomaly_detection_features_indices(specific_features)
                similarity, most_similar_vector = anomaly_detection.get_similarity(features_vector)
                similarities.append(similarity)
                if operational_config.is_raw_data_send_enabled():
                    send_raw_data({
                        **metrics_dict, 
                        'similarity' : similarity, 
                        'is_anomaly' : bool(similarity < anomaly_detection.get_similarity_threshold()),
                        'time' : datetime.datetime.now().strftime("%H:%M:%S")
                    })
            
            if mode & MODE.DETECTION_THRESHOLD_CALIBRATION:
                if similarity < anomaly_detection.get_similarity_threshold() and similarity > 0:
                    # similarity > 0 is used because watchpoints collected at previously unseen PC
                    # will result in 0 similarity, and it shouldn't be used as threshold
                    anomaly_detection.set_similarity_threshold(similarity)
                    print(f'DETECTION_THRESHOLD_CALIBRATION: Updated similarity threshold to {anomaly_detection.get_similarity_threshold()}')
                    send_similarity_threshold(anomaly_detection.get_similarity_threshold())
                
            if mode & MODE.TESTING:
                if similarity != 1:
                    print(f'Similarity of the following performance counters were not 1 ({similarity}):')
                    for event_name in event_names:
                        print(f'{event_name:<13}', end='')
                    print()
                    for v in features_vector:
                        print(f'{v:<13}', end='')
                    print()
                    if most_similar_vector is not None:
                        print(f'Most similar vector:')
                        for v in most_similar_vector:
                            print(f'{v:<13}', end='')
                    print('\n')
            
                if similarity < anomaly_detection.get_similarity_threshold():
                    send_new_anomaly(metrics_dict, similarity, features_vector, most_similar_vector)
                        
            if mode & MODE.TRAINING:
                anomaly_detection.update_dataset(features_vector)
            
            
            
            total_clk_time += metrics_dict["clk_counter"]
            total_halted_time += metrics_dict["fifo_full_ticks_count"]
            
            if mode != MODE.IDLE:
                clk_time_since_last_send += metrics_dict["clk_counter"]
                halted_time_since_last_send += metrics_dict["fifo_full_ticks_count"]
#                 items_since_last_send += 1           
            
            start += chunks_per_item
            end += chunks_per_item
        items_transferred = 0
        if print_stats:
            print(f'dataset size = {anomaly_detection.get_dataset_size()}', end=', ')
            print(f'processing time: {time.time() - processing_time_checkpoint}s')
#         time.sleep(1)

operate_thread = Thread(target=operate, daemon=True)
operate_thread.start()

#print(f'Total clk_count = {total_clk_time / CLK_SPEED}s')
#print(f'Total halted_time = {total_halted_time / CLK_SPEED}s')

In [7]:
# print_stats = True

In [8]:
# end_operate_thread = True

In [9]:
# s = console_io.read()
# for line in s.split('\n'):
#     print(line)

In [10]:
# for row in anomaly_detection.dataset:
#     for val in row:
#         print(int(val), end=', ')
#     print()

In [11]:
# anomaly_detection.max_interval
# anomaly_detection.get_lack_of_matches_threshold()

In [12]:
anomaly_detection.dataset

{'default': None}

In [13]:
anomaly_detection.dataset

{'default': None}

In [14]:
anomaly_detection.get_used_features()

{'Core__BRANCH': True,
 'Core__JAL': True,
 'Core__LOAD': True,
 'Core__STORE': True,
 'L1I__LD': True,
 'L1D__LD': True,
 'TGC__WRITE': True,
 'TGC__READ': True,
 'HPC_overflow_map': True,
 'pc': True,
 'clk_counter': True,
 'instr': True,
 'fifo_full_ticks_count': True,
 'A0': True,
 'A1': True,
 'A2': True,
 'A3': True,
 'feature_extractor_result': True,
 'cumulative_xor_pc': True}

In [15]:
anomaly_detection.dataset

{'default': None}

In [16]:
anomaly_detection.special_columns_indices

[9]

New connection at ID 0 ('192.168.0.105', 39891)
Calling rpc_list_programs with args=[]
Calling rpc_update_status with args=[]
Calling rpc_set_operational_config with args=['{"raw_data_send_enable":true}']
Saving config:  {'disable_saving': False, 'f_name': 'config.pickle', 'periodic_send_interval_seconds': 0.8, 'items_collected_processing_limit': 500, 'lack_of_matches_threshold_multiplier_of_max_interval': 1.5, 'raw_data_send_enable': True}
Calling rpc_load_program with args=['echo']
Calling rpc_set_feature_enabled with args=['instr', 'false']
Calling rpc_set_feature_enabled with args=['fifo_full_ticks_count', 'false']
Calling rpc_set_feature_enabled with args=['feature_extractor_result', 'false']
Calling rpc_set_feature_enabled with args=['cumulative_xor_pc', 'false']
Calling rpc_set_feature_enabled with args=['clk_counter', 'false']
Calling rpc_set_feature_enabled with args=['TGC__WRITE', 'false']
Calling rpc_set_feature_enabled with args=['TGC__READ', 'false']
Calling rpc_set_featur

Calling rpc_reset_dataset with args=[]
Calling rpc_set_feature_enabled with args=['A2', 'true']
Calling rpc_set_feature_enabled with args=['A3', 'true']
Calling rpc_set_feature_enabled with args=['A3', 'false']
Calling rpc_set_feature_enabled with args=['A2', 'false']
Calling rpc_set_feature_enabled with args=['A2', 'true']
Calling rpc_set_feature_enabled with args=['A2', 'false']
Calling rpc_set_feature_enabled with args=['A2', 'true']
Calling rpc_send_stdin with args=['abc\n']
abc
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__WRITE': False, 'TGC__READ': False, 'HPC_overflow_map': False, 'pc': True, 'clk_counter': False, 'instr': False, 'fifo_full_ticks_count': False, 'A0': True, 'A1': True, 'A2': True, 'A3': False, 'feature_extractor_result': False, 'cumulative_xor_pc': False}
[2147483900, 97, 2147544111, 1]
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD

Calling rpc_send_stdin with args=['A\n']
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__WRITE': False, 'TGC__READ': False, 'HPC_overflow_map': False, 'pc': True, 'clk_counter': False, 'instr': False, 'fifo_full_ticks_count': False, 'A0': True, 'A1': True, 'A2': True, 'A3': False, 'feature_extractor_result': False, 'cumulative_xor_pc': False}
[2147483900, 65, 2147544111, 1]
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__WRITE': False, 'TGC__READ': False, 'HPC_overflow_map': False, 'pc': True, 'clk_counter': False, 'instr': False, 'fifo_full_ticks_count': False, 'A0': True, 'A1': True, 'A2': True, 'A3': False, 'feature_extractor_result': False, 'cumulative_xor_pc': False}
[2147483900, 10, 2147544111, 1]
A
Calling rpc_send_stdin with args=['0\n']
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': Fa

Error sending data to client 4: [Errno 32] Broken pipe
New connection at ID 5 ('192.168.0.105', 4821)
Calling rpc_list_programs with args=[]
Calling rpc_update_status with args=[]
Calling rpc_set_operational_config with args=['{"raw_data_send_enable":true}']
Saving config:  {'disable_saving': False, 'f_name': 'config.pickle', 'periodic_send_interval_seconds': 0.8, 'items_collected_processing_limit': 500, 'lack_of_matches_threshold_multiplier_of_max_interval': 1.5, 'raw_data_send_enable': True}
Calling rpc_send_stdin with args=['1234\n']
1234
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__WRITE': False, 'TGC__READ': False, 'HPC_overflow_map': False, 'pc': True, 'clk_counter': False, 'instr': False, 'fifo_full_ticks_count': False, 'A0': True, 'A1': True, 'A2': True, 'A3': False, 'feature_extractor_result': False, 'cumulative_xor_pc': False}
[2147483900, 49, 2147544111, 1]
{'Core__BRANCH': False, 'Core__JAL'

...

[2147483900, 0, 2147544111, 1, 1]
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__WRITE': False, 'TGC__READ': False, 'HPC_overflow_map': False, 'pc': True, 'clk_counter': False, 'instr': False, 'fifo_full_ticks_count': False, 'A0': True, 'A1': True, 'A2': True, 'A3': True, 'feature_extractor_result': False, 'cumulative_xor_pc': False}
[2147483900, 0, 2147544111, 1, 1]
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__WRITE': False, 'TGC__READ': False, 'HPC_overflow_map': False, 'pc': True, 'clk_counter': False, 'instr': False, 'fifo_full_ticks_count': False, 'A0': True, 'A1': True, 'A2': True, 'A3': True, 'feature_extractor_result': False, 'cumulative_xor_pc': False}
[2147483900, 0, 2147544111, 1, 1]
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__W

[2147483900, 0, 2147544111, 1, 1]
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__WRITE': False, 'TGC__READ': False, 'HPC_overflow_map': False, 'pc': True, 'clk_counter': False, 'instr': False, 'fifo_full_ticks_count': False, 'A0': True, 'A1': True, 'A2': True, 'A3': True, 'feature_extractor_result': False, 'cumulative_xor_pc': False}
[2147483900, 0, 2147544111, 1, 1]
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__WRITE': False, 'TGC__READ': False, 'HPC_overflow_map': False, 'pc': True, 'clk_counter': False, 'instr': False, 'fifo_full_ticks_count': False, 'A0': True, 'A1': True, 'A2': True, 'A3': True, 'feature_extractor_result': False, 'cumulative_xor_pc': False}
[2147483900, 0, 2147544111, 1, 1]
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__W

{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__WRITE': False, 'TGC__READ': False, 'HPC_overflow_map': False, 'pc': True, 'clk_counter': False, 'instr': False, 'fifo_full_ticks_count': False, 'A0': True, 'A1': True, 'A2': True, 'A3': True, 'feature_extractor_result': False, 'cumulative_xor_pc': False}
[2147483900, 0, 2147544111, 1, 1]
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__WRITE': False, 'TGC__READ': False, 'HPC_overflow_map': False, 'pc': True, 'clk_counter': False, 'instr': False, 'fifo_full_ticks_count': False, 'A0': True, 'A1': True, 'A2': True, 'A3': True, 'feature_extractor_result': False, 'cumulative_xor_pc': False}
[2147483900, 0, 2147544111, 1, 1]
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__WRITE': False, 'TGC__READ': False, 

Calling rpc_send_stdin with args=['\n']
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

Calling rpc_send_stdin with args=['A\n']
A
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__WRITE': False, 'TGC__READ': False, 'HPC_overflow_map': False, 'pc': True, 'clk_counter': False, 'instr': False, 'fifo_full_ticks_count': False, 'A0': True, 'A1': True, 'A2': True, 'A3': True, 'feature_extractor_result': False, 'cumulative_xor_pc': False}
[2147483900, 65, 2147544111, 1, 1]
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE': False, 'L1I__LD': False, 'L1D__LD': False, 'TGC__WRITE': False, 'TGC__READ': False, 'HPC_overflow_map': False, 'pc': True, 'clk_counter': False, 'instr': False, 'fifo_full_ticks_count': False, 'A0': True, 'A1': True, 'A2': True, 'A3': True, 'feature_extractor_result': False, 'cumulative_xor_pc': False}
[2147483900, 10, 2147544111, 1, 1]
Calling rpc_send_stdin with args=['S\n']
{'Core__BRANCH': False, 'Core__JAL': False, 'Core__LOAD': False, 'Core__STORE'